In [1]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import polars as pl

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)
from deeplifting.utils import create_optimization_plot

In [3]:
# fig = create_optimization_plot(problem_name, problem, results, colormap='autumn_r')

In [4]:
# fig.savefig(
#     '/Users/ryandevera/data-science/umn_environments/Deeplifting'
#     f'/images/{problem_name}_surface_and_contour-autumn.png'
# )

In [5]:
# Run all problems
problem_names = [
    'ackley',
    'bukin_n6',
    'cross_in_tray',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'ex8_1_1',
    # 'ex8_6_2',  # Need to fix threw an error
    'kriging_peaks_red010',
    'kriging_peaks_red020',
    'kriging_peaks_red030',
    'kriging_peaks_red050',
    'kriging_peaks_red100',
    'kriging_peaks_red200',
    'kriging_peaks_red500',
    'mathopt6',
    'quantum',
    'rosenbrock',
]

problem_performance = []
trials = 10

for problem_name in problem_names:
    problem = PROBLEMS_BY_NAME[problem_name]
    outputs = run_deeplifting(problem, trials=trials)
    results = outputs['final_results']

    # get the global minimum and the number of hits
    minimum_value = problem['global_minimum']
    hit = np.array([np.abs(found_min - minimum_value) for _, _, found_min in results])
    hits = np.where(hit <= 1e-4, 1, 0).mean()

    # Create a performance dataframe
    df = pl.DataFrame(
        {
            'problem_name': [problem_name] * trials,
            'algorithm': ['deeplifting'] * trials,
            'hits': hits,
        }
    )
    problem_performance.append(df)

n_gpus: 0
cpu


╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
══════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                          ║ 
Version 1.2.0                                                                                 ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang               ║ 
══════════════════════════════════════════════════════════════════════════════════════════════╣
Problem spe

/Users/ryandevera/.virtualenvs/deeplifing/lib/python3.9/site-packages/pygranso/private/bfgsHessianInverseLimitedMem.py:237: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3575.)
  alpha[j,:]  = self.rho[0,j] * (self.S[:,j].T  @ q)


   2 ║  - │   -   ║ -2.06252434067 ║   -  │   -  ║ QN │     9 │ 0.003906 ║     1 │ 9.34e-04   ║ 
   3 ║  - │   -   ║ -2.06252480004 ║   -  │   -  ║ QN │     8 │ 0.007812 ║     1 │ 1.39e-04   ║ 
   4 ║  - │   -   ║ -2.06260864155 ║   -  │   -  ║ QN │     1 │ 1.000000 ║     1 │ 5.75e-07   ║ 
   5 ║  - │   -   ║ -2.06261187420 ║   -  │   -  ║ QN │     1 │ 1.000000 ║     1 │ 3.03e-09   ║ 
   6 ║  - │   -   ║ -2.06261187431 ║   -  │   -  ║ QN │     1 │ 1.000000 ║     1 │ 1.11e-11   ║ 
═════╩════════════╩════════════════╩═════════════╩═══════════════════════╩════════════════════╣
F = final iterate, B = Best (to tolerance), MF = Most Feasible                                ║ 
Optimization results:                                                                         ║ 
═════╦════════════╦════════════════╦═════════════╦═══════════════════════╦════════════════════╣
   F ║    │       ║ -2.06261187431 ║   -  │   -  ║    │       │          ║       │            ║ 
   B ║    │       ║ -2.062611874


KeyboardInterrupt



In [6]:
performance_df = pl.concat(problem_performance, how='vertical')

In [10]:
performance_df = (
    performance_df
    .groupby(['problem_name'])
    .agg(pl.col('hits').mean())
    .rename({'hits': 'success_rate'})
    .sort('problem_name')
)

In [13]:
performance_df

problem_name,success_rate
str,f64
"""ackley""",1.0
"""bukin_n6""",1.0
"""cross_in_tray""",1.0
"""drop_wave""",1.0
"""eggholder""",0.6
"""ex8_1_1""",1.0
"""griewank""",1.0
"""holder_table""",1.0
"""kriging_peaks_…",1.0
